# Pre-requisites

In [68]:
import numpy as np
import pdb
import itertools as itt

# Problem Formulation:

In [69]:
# Line matrix:
# Format:            Starting_Node  Ending_Node       R         X
Linedata1 = np.array([    0,             1,         0.02,     0.04,
                          0,             2,         0.01,     0.03,
                          1,             0,         0.02,     0.04,
                          1,             2,         0.0125,   0.025,
                          2,             0,         0.01,     0.03,
                          2,             1,         0.0125,   0.025]).reshape(6,4);

j = np.sqrt(-1+0j); S_base = 100;
# Load matrix: Status is 0 for slack, 1 for PQ buses, and 2 for PV buses.
# Format:              Bus_Number  Status_of_Buses     Voltage (pu)   Real_Power_injected    Reactive_Power_injected
Loaddata1 = np.array([   0,             0,                 1.05,                 0,                          0,
                         1,             1,                 1.00,                -400,                        -250,
                         2,             2,                 1.04,                 200,                      0]).reshape(3,5);

In [70]:
P1 = np.copy(Loaddata1[:,3])/S_base;
P1_cal = np.zeros(len(Loaddata1[:,0]));
Q1 = np.copy(Loaddata1[:,4])/S_base;
Q1_cal = np.zeros(len(Loaddata1[:,0]));
Q1_old = np.copy(Loaddata1[:,4])/S_base;
V1 = np.copy(Loaddata1[:,2]);
V1_old = np.copy(Loaddata1[:,2]);
Delta1 = np.zeros(len(Loaddata1[:,0]));
Jacobian = np.zeros([len(Loaddata1[:,0]),len(Loaddata1[:,0])])

print(V1)

[1.05 1.   1.04]


# Y bus matrix formulation

In [71]:
def common_member(a,b):
  a_set = set(a)
  b_set = set(b)

  # check length
  if len(a_set.intersection(b_set)) > 0:
      return(list(a_set.intersection(b_set)))
  else:
      return([])

Y1= np.zeros([len(Loaddata1[:,0]),len(Loaddata1[:,0])],dtype=np.complex_);
for i,k in zip(list(Linedata1[:,0]),list(Linedata1[:,1])):
  i = int(i);k=int(k)
  # print(f"This is i: {i} & this is k: {k}")
  Y1[i,k] += -1*(1/(Linedata1[common_member(np.where(Linedata1[:,0]==i)[0],np.where(Linedata1[:,1]==k)[0]),2]+j*Linedata1[common_member(np.where(Linedata1[:,0]==i)[0],np.where(Linedata1[:,1]==k)[0]),3]))[0]
for i in Loaddata1[:,0]:
  i = int(i);
  Y1[i,i] += sum(1/(Linedata1[pp,2]+j*Linedata1[pp,3]) for pp in list(np.where(Linedata1[:,0]==i)[0]))
G = np.real(Y1)
B = np.imag(Y1)
print(Y1)
print(G)
print(B)

[[ 20.-50.j -10.+20.j -10.+30.j]
 [-10.+20.j  26.-52.j -16.+32.j]
 [-10.+30.j -16.+32.j  26.-62.j]]
[[ 20. -10. -10.]
 [-10.  26. -16.]
 [-10. -16.  26.]]
[[-50.  20.  30.]
 [ 20. -52.  32.]
 [ 30.  32. -62.]]


# Standard Newton Raphson

## Power calculations
Explaination_by_Yash: I mean... could we be any more direct? We just calculated power for each bus except the slack bus and then the corresponding daviations of power.

In [72]:
for i in Loaddata1[:,0]:
  i = int(i)
  if Loaddata1[i,1] != 0:
    P1_cal[i] = abs(V1[i])*sum([V1[int(k)]*(G[i,int(k)]*np.cos(Delta1[i]-Delta1[int(k)])+B[i,int(k)]*np.sin(Delta1[i]-Delta1[int(k)])) for k in Loaddata1[:,0]])
  if Loaddata1[i,1] == 1:
    Q1_cal[i] = abs(V1[i])*sum([V1[int(k)]*(G[i,int(k)]*np.sin(Delta1[i]-Delta1[int(k)])-B[i,int(k)]*np.cos(Delta1[i]-Delta1[int(k)])) for k in Loaddata1[:,0]])
Del_P1 = P1-P1_cal;
Del_Q1 = Q1-Q1_cal;
delY = np.append(np.copy(Del_P1[1:]),(Del_Q1[np.where(Loaddata1[:,1] == 1)]))
# print(P1_cal)
# print(Q1_cal)
# print(Del_P1)
# print(Del_Q1)
print(delY)

[-2.86    1.4384 -0.22  ]


## Jacobian Computation


### H calculation delP/delDelta
We know that H is delP/delDelta however, we calculate the Delta for everybus except for slack bus and same goes for P. Hence dimension of H will be (N-1)*(N-1)

In [73]:
H = np.zeros([len(Loaddata1[:,0]),len(Loaddata1[:,0])])
for i,k in itt.product(Loaddata1[:,0],Loaddata1[:,0]):
  i = int(i); k = int(k);
  if Loaddata1[k,1] != 0 and Loaddata1[i,1] !=0:
    # print(f"This is i: {i} & this is k: {k}")
    if i==k:
      H[i,k] = -np.real(Q1[i])-B[i,i]*(abs(V1[i])**2);
    else:
      H[i,k] =  abs(V1[i])*abs(V1[k])*(G[i,k]*np.sin(Delta1[i]-Delta1[k])-B[i,k]*np.cos(Delta1[i]-Delta1[k]))
print(H)
# print(Delta1)
# print(B)
# print(G)
# print(np.real(Q1))

[[  0.       0.       0.    ]
 [  0.      54.5    -33.28  ]
 [  0.     -33.28    67.0592]]


### N calculation V*delP/delV
We know that N is VdelP/delV however, we calculate the V for PQ busses and same goes for P. Hence dimension of H will be (NPq)*(NPq)

In [74]:
N = np.zeros([len(Loaddata1[:,0]),len(Loaddata1[:,0])])
for i,k in itt.product(Loaddata1[:,0],Loaddata1[:,0]):
  i = int(i); k = int(k);
  if Loaddata1[k,1] == 1 and Loaddata1[i,1] != 0:
    # print(f"This is i: {i} & this is k: {k}")
    if i==k:
      N[i,k] = np.real(P1[i])+G[i,i]*(abs(V1[i])**2);
    else:
      N[i,k] =  abs(V1[i])*abs(V1[k])*(G[i,k]*np.cos(Delta1[i]-Delta1[k])+B[i,k]*np.sin(Delta1[i]-Delta1[k]))
print(N)
# print(np.real(P1))

[[  0.     0.     0.  ]
 [  0.    22.     0.  ]
 [  0.   -16.64   0.  ]]


### J calculation delQ/delDelta
Now Q is there for PV buses and Delta for every bus except the slack bus. Hence, our dimension for J would be (Npv)*(N-1)

In [75]:
J = np.zeros([len(Loaddata1[:,0]),len(Loaddata1[:,0])])
for i,k in itt.product(Loaddata1[:,0],Loaddata1[:,0]):
  i = int(i); k = int(k);
  if Loaddata1[i,1] == 1 and Loaddata1[k,1] != 0:
    # print(f"This is i: {i} & this is k: {k}")
    if i==k:
      J[i,k] = np.real(P1[i])-G[i,i]*(abs(V1[i])**2);
    else:
      J[i,k] =  abs(V1[i])*abs(V1[k])*(-G[i,k]*np.cos(Delta1[i]-Delta1[k])-B[i,k]*np.sin(Delta1[i]-Delta1[k]))
print(J)
# print(np.real(P1))

[[  0.     0.     0.  ]
 [  0.   -30.    16.64]
 [  0.     0.     0.  ]]


### L calculation V*delQ/delV
The Q is for PV buses and V is for PQ buses. Hence, the dimension of L will be (Npv)*(Npq).

In [76]:
L = np.zeros([len(Loaddata1[:,0]),len(Loaddata1[:,0])])
for i,k in itt.product(Loaddata1[:,0],Loaddata1[:,0]):
  i = int(i); k = int(k);
  if Loaddata1[i,1] == 1 and Loaddata1[k,1] == 1:
    # print(f"This is i: {i} & this is k: {k}")
    if i==k:
      L[i,k] = np.real(Q1[i])-B[i,i]*(abs(V1[i])**2);
    else:
      L[i,k] =  abs(V1[i])*abs(V1[k])*(G[i,k]*np.sin(Delta1[i]-Delta1[k])-B[i,k]*np.cos(Delta1[i]-Delta1[k]))
print(L)
# print(np.real(Q1))

[[ 0.   0.   0. ]
 [ 0.  49.5  0. ]
 [ 0.   0.   0. ]]


### Forming the Jacobian:

In [77]:
Jacobian[:len(Loaddata1[:,0])-1,:len(Loaddata1[:,0])-1] = H[1:len(Loaddata1[:,0]),1:len(Loaddata1[:,0])]
Jacobian[:len(Loaddata1[:,0])-1,np.where(Loaddata1[:,1]==2)[0]] = N[1:len(Loaddata1[:,0]),np.where(Loaddata1[:,1]==1)[0]]
Jacobian[np.where(Loaddata1[:,1]==1)[0],:len(Loaddata1[:,0])-1] = J[np.where(Loaddata1[:,1]==1)[0],1:len(Loaddata1[:,0])]
# Jacobian[np.where(Loaddata1[:,1]==1)[0],np.where(Loaddata1[:,1]==1)[0]] = L[np.where(Loaddata1[:,1]==1)[0],np.where(Loaddata1[:,1]==1)[0]]
Jacobian[np.where(Loaddata1[:,1]==1)[0],np.where(Loaddata1[:,1]==1)[0]] = 0

# print(np.where(Loaddata1[:,1]==2)[0])
print(Jacobian)

[[ 54.5  -33.28  22.  ]
 [-30.     0.   -16.64]
 [  0.     0.     0.  ]]


In [78]:
# Tikdham:
Jacobian = np.zeros([3,3])
Jacobian[0:2,0:2] = H[1:3,1:3]
Jacobian[0:2,2] = N[1:3,1]
Jacobian[2,0:2] = J[1,1:3]
Jacobian[2,2] = L[1,1]
print(Jacobian)



[[ 54.5    -33.28    22.    ]
 [-33.28    67.0592 -16.64  ]
 [-30.      16.64    49.5   ]]


## delDelta and delV/V calculations and updation of Delta and V

In [79]:
# delY =
delX = np.matmul(np.linalg.inv(Jacobian),delY)
# print(np.linalg.inv(Jacobian))
print(delX)

[-0.0458804  -0.00860464 -0.0293582 ]


In [80]:
print(Delta1)
print(V1)
Delta1[np.where(Loaddata1[:,1]!=0)[0]] += delX[:len(np.where(Loaddata1[:,1]!=0)[0])]
V1[np.where(Loaddata1[:,1]==1)[0]] += delX[np.where(Loaddata1[:,1]==1)[0]]*V1[np.where(Loaddata1[:,1]==1)[0]]
print(Delta1)
print(V1)

[0. 0. 0.]
[1.05 1.   1.04]
[ 0.         -0.0458804  -0.00860464]
[1.05       0.99139536 1.04      ]


## Generalised Class for easy computation
Explaination_by_Yash: Since we have already seen how to do the load flow equations and daviation updations. Lets directly jump to the class formation like we did in gauss siedel. Moreover...I'm lazy to analyse again and again lol! XD XD

I know I know thodha complex hogaya hai but we will get through it. One equation at a time.

In [81]:
class Std_NR():
  def __init__(self, Linedata1, Loaddata1, N=2, Epsilon=1e-5, S_base=100):
    self.Linedata1=Linedata1;
    self.Loaddata1=Loaddata1;
    self.N = N;
    self.Epsilon = Epsilon;
    self.S_base = 100; self.j = np.sqrt(-1+0j);
    self.Vmin = 0.95; self.Vmax = 1.05;
    self.Qmin = -1.0; self.Qmax = 1.0;
    self.P1 = np.copy(self.Loaddata1[:,3])/self.S_base;
    self.P1_cal = np.zeros(len(self.Loaddata1[:,0]));
    self.Q1 = np.copy(self.Loaddata1[:,4])/self.S_base;
    self.Q1_cal = np.zeros(len(self.Loaddata1[:,0]));
    self.Q1_old = np.copy(self.Loaddata1[:,4])/self.S_base;
    self.V1 = np.copy(self.Loaddata1[:,2]);
    self.V1_old = np.copy(self.Loaddata1[:,2]);
    self.Delta1 = np.zeros(len(self.Loaddata1[:,0]));
    self.Jacobian = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    self.I = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);
    self.S = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);
    self.S_loss = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);

  def common_member(self,a,b):
    a_set = set(a)
    b_set = set(b)

    # check length
    if len(a_set.intersection(b_set)) > 0:
        return(list(a_set.intersection(b_set)))
    else:
        return([])

  def Y_bus(self,Linedata1, Loaddata1):
    Y1= np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])],dtype=np.complex_);
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k)
      # print(f"This is i: {i} & this is k: {k}")
      Y1[i,k] += -1*(1/(self.Linedata1[self.common_member(np.where(self.Linedata1[:,0]==i)[0],np.where(self.Linedata1[:,1]==k)[0]),2]+self.j*self.Linedata1[self.common_member(np.where(self.Linedata1[:,0]==i)[0],np.where(self.Linedata1[:,1]==k)[0]),3]))[0]
    for i in self.Loaddata1[:,0]:
      i = int(i);
      Y1[i,i] += sum(1/(self.Linedata1[pp,2]+self.j*self.Linedata1[pp,3]) for pp in list(np.where(self.Linedata1[:,0]==i)[0]))
    G = np.real(Y1)
    B = np.imag(Y1)
    return G,B,Y1

  def Jacobian_Calculations(self):
    # H:
    self.H = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    for i,k in itt.product(self.Loaddata1[:,0],self.Loaddata1[:,0]):
      i = int(i); k = int(k);
      if self.Loaddata1[k,1] != 0 and self.Loaddata1[i,1] !=0:
        if i==k:
          self.H[i,k] = -np.real(self.Q1[i])-self.B[i,i]*(abs(self.V1[i])**2);
        else:
          self.H[i,k] =  abs(self.V1[i])*abs(self.V1[k])*(self.G[i,k]*np.sin(self.Delta1[i]-self.Delta1[k])-self.B[i,k]*np.cos(self.Delta1[i]-self.Delta1[k]))
    # N:
    self.N_ = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    for i,k in itt.product(self.Loaddata1[:,0],self.Loaddata1[:,0]):
      i = int(i); k = int(k);
      if self.Loaddata1[k,1] == 1 and self.Loaddata1[i,1] != 0:
        if i==k:
          self.N_[i,k] = np.real(self.P1[i])+self.G[i,i]*(abs(self.V1[i])**2);
        else:
          self.N_[i,k] =  abs(self.V1[i])*abs(self.V1[k])*(self.G[i,k]*np.cos(self.Delta1[i]-self.Delta1[k])+self.B[i,k]*np.sin(self.Delta1[i]-self.Delta1[k]))
    # J:
    self.J = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    for i,k in itt.product(self.Loaddata1[:,0],self.Loaddata1[:,0]):
      i = int(i); k = int(k);
      if self.Loaddata1[i,1] == 1 and self.Loaddata1[k,1] != 0:
        if i==k:
          self.J[i,k] = np.real(self.P1[i])-self.G[i,i]*(abs(self.V1[i])**2);
        else:
          self.J[i,k] =  abs(self.V1[i])*abs(self.V1[k])*(-self.G[i,k]*np.cos(self.Delta1[i]-self.Delta1[k])-self.B[i,k]*np.sin(self.Delta1[i]-self.Delta1[k]))
    # L:
    self.L = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    for i,k in itt.product(self.Loaddata1[:,0],self.Loaddata1[:,0]):
      i = int(i); k = int(k);
      if self.Loaddata1[i,1] == 1 and self.Loaddata1[k,1] == 1:
        if i==k:
          self.L[i,k] = np.real(self.Q1[i])-self.B[i,i]*(abs(self.V1[i])**2);
        else:
          self.L[i,k] =  abs(self.V1[i])*abs(self.V1[k])*(self.G[i,k]*np.sin(self.Delta1[i]-self.Delta1[k])-self.B[i,k]*np.cos(self.Delta1[i]-self.Delta1[k]))
    # Formulation:
    # self.Jacobian[:len(self.Loaddata1[:,0])-1,:len(self.Loaddata1[:,0])-1] = self.H[1:len(self.Loaddata1[:,0]),1:len(self.Loaddata1[:,0])]
    # self.Jacobian[:len(self.Loaddata1[:,0])-1,np.where(self.Loaddata1[:,1]==1)[0]] = self.N[1:len(self.Loaddata1[:,0]),np.where(self.Loaddata1[:,1]==1)[0]]
    # self.Jacobian[np.where(self.Loaddata1[:,1]==1)[0],:len(self.Loaddata1[:,0])-1] = self.J[np.where(self.Loaddata1[:,1]==1)[0],1:len(self.Loaddata1[:,0])]
    # self.Jacobian[np.where(self.Loaddata1[:,1]==1)[0],np.where(self.Loaddata1[:,1]==1)[0]] = self.L[np.where(self.Loaddata1[:,1]==1)[0],np.where(self.Loaddata1[:,1]==1)[0]]
    self.Jacobian[0:2,0:2] = self.H[1:3,1:3]
    self.Jacobian[0:2,2] = self.N_[1:3,1]
    self.Jacobian[2,0:2] = self.J[1,1:3]
    self.Jacobian[2,2] = self.L[1,1]
    
    return self.Jacobian

  def Calculations(self):
    self.G,self.B,self.Y1 = self.Y_bus(self.Linedata1,self.Loaddata1);
    Convergence = {};
    for ii in range(self.N):
      # print(f"Yeh hai ii: {ii}")
      for i in self.Loaddata1[:,0]:
        i = int(i)
        if self.Loaddata1[i,1] != 0:
          self.P1_cal[i] = abs(self.V1[i])*sum([self.V1[int(k)]*(self.G[i,int(k)]*np.cos(self.Delta1[i]-self.Delta1[int(k)])+self.B[i,int(k)]*np.sin(self.Delta1[i]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
        if self.Loaddata1[i,1] == 1:
          self.Q1_cal[i] = abs(self.V1[i])*sum([self.V1[int(k)]*(self.G[i,int(k)]*np.sin(self.Delta1[i]-self.Delta1[int(k)])-self.B[i,int(k)]*np.cos(self.Delta1[i]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
      self.Q1_cal[self.Q1_cal<=self.Qmin]=self.Qmin;
      self.Q1_cal[self.Q1_cal>=self.Qmax]=self.Qmax;
      self.Del_P1 = np.copy(self.P1)-np.copy(self.P1_cal);
      self.Del_Q1 = np.copy(self.Q1)-np.copy(self.Q1_cal);
      # print("Our Epsilon: ",self.Epsilon)
      # print(abs(self.Del_P1))
      # print(abs(self.Del_Q1))
      # print("np.all(abs(Del_P1[1:]) < self.Epsilon): ",np.all(abs(self.Del_P1[1:]) < self.Epsilon))
      # print("abs(Del_P1[1:]) < self.Epsilon: ",abs(self.Del_P1[1:]) < self.Epsilon)
      if np.all(abs(self.Del_P1[1:]) < self.Epsilon) and np.all(abs(self.Del_Q1[1:]) < self.Epsilon): #and np.all(abs(Del_Q1[1:]))<= self.Epsilon:
        # print(f"Yeh hai ii during break: {ii}")
        break
      # print("Reaching")
      self.delY = np.append(np.copy(self.Del_P1[1:]),(self.Del_Q1[np.where(self.Loaddata1[:,1] == 1)]))
      self.Jacobian = self.Jacobian_Calculations()
      self.delX = np.matmul(np.linalg.inv(self.Jacobian),self.delY)
      self.Delta1[np.where(self.Loaddata1[:,1]!=0)[0]] += self.delX[:len(np.where(self.Loaddata1[:,1]!=0)[0])]
      self.V1[np.where(self.Loaddata1[:,1]==1)[0]] += self.delX[np.where(self.Loaddata1[:,1]==1)[0]]*self.V1[np.where(self.Loaddata1[:,1]==1)[0]]
      self.V1[self.V1<=self.Vmin]=self.Vmin;
      self.V1[self.V1>=self.Vmax]=self.Vmax;
      Convergence[ii] = [(self.Del_P1[1:]), (self.Del_Q1[1:])];
    # print("Baba re baba agaya bahar on iteration: ",ii+1)

    # Slack bus computations:
    self.P1_cal[0] = abs(self.V1[0])*sum([self.V1[int(k)]*(self.G[0,int(k)]*np.cos(self.Delta1[0]-self.Delta1[int(k)])+self.B[0,int(k)]*np.sin(self.Delta1[0]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
    self.Q1_cal[0] = abs(self.V1[0])*sum([self.V1[int(k)]*(self.G[0,int(k)]*np.sin(self.Delta1[0]-self.Delta1[int(k)])-self.B[0,int(k)]*np.cos(self.Delta1[0]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])

    # Line Flows:
    ## Branch Currents:
    self.V1c = np.copy(self.V1)*(np.cos(self.Delta1)+self.j*np.sin(self.Delta1));
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.I[i,k] = (self.V1c[i]-self.V1c[k])*self.Y1[i,k];

    ## Branch Flow:
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.S[i,k] = (self.V1c[i])*np.conjugate((self.V1c[i]-self.V1c[k])*self.Y1[i,k]);

    ## Branch losses:
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.S_loss[i,k] = self.S[i,k]-self.S[k,i];

    return self.V1, self.Delta1, self.P1_cal, self.Q1_cal, self.Jacobian, self.I, self.S, self.S_loss, ii+1, self.V1c, self.Del_P1, self.Del_Q1, Convergence

In [88]:
NR = Std_NR(Linedata1, Loaddata1,N=2,Epsilon=1e-14)
Y = NR.Y_bus(Linedata1, Loaddata1)
V1, Delta1, P1_cal, Q1_cal, Jacobian, I, S, S_loss, N, V1c, Del_P1, Del_Q1, dir_Convergence = NR.Calculations()
print("V1: ",V1)
print("Delta1: ",Delta1)
print("P1_cal: ",P1_cal)
print("Q1_cal: ",Q1_cal)
print("Jacobian: ",Jacobian)
print("I: ",I)
print("S: ",S)
print("S_loss: ",S_loss)
print("N: ",N)
print("self.V1c: ",V1c)
print("Del_P1: ",Del_P1)
print("Del_Q1: ",Del_Q1)
print("abs(Del_P1): ",abs(Del_P1))
print("abs(Del_Q1): ",abs(Del_Q1))
print(dir_Convergence)

V1:  [1.05       0.99059171 1.04      ]
Delta1:  [ 0.         -0.04146653 -0.00940196]
P1_cal:  [ 1.90857173 -3.10244563  1.33986665]
Q1_cal:  [ 1.04805745 -1.          0.        ]
Jacobian:  [[ 53.46186388 -32.46649821  21.48093194]
 [-33.39932783  67.0592     -15.53362689]
 [-29.48093194  17.39928613  48.46186388]]
I:  [[ 0.         -1.42389079 -0.39379657]
 [ 1.42389079  0.          1.80459711]
 [ 0.39379657 -1.80459711  0.        ]]
S:  [[ 0.         -1.49508533 -0.4134864 ]
 [ 1.44190984  0.          1.83151025]
 [ 0.41152111 -1.88751489  0.        ]]
S_loss:  [[ 0.         -2.93699517 -0.82500751]
 [ 2.93699517  0.          3.71902514]
 [ 0.82500751 -3.71902514  0.        ]]
N:  2
self.V1c:  [1.05      +0.j         0.98974019-0.04106463j 1.03995403-0.0097779j ]
Del_P1:  [ 0.         -0.89755437  0.66013335]
Del_Q1:  [ 0.  -1.5  0. ]
abs(Del_P1):  [0.         0.89755437 0.66013335]
abs(Del_Q1):  [0.  1.5 0. ]
{0: [array([-2.86  ,  1.4384]), array([-1.5,  0. ])], 1: [array([-0.8975

C:\Users\y.raghuwanshi\AppData\Local\Temp\ipykernel_11852\4008556997.py:138: ComplexWarning: Casting complex values to real discards the imaginary part
  self.I[i,k] = (self.V1c[i]-self.V1c[k])*self.Y1[i,k];
C:\Users\y.raghuwanshi\AppData\Local\Temp\ipykernel_11852\4008556997.py:143: ComplexWarning: Casting complex values to real discards the imaginary part
  self.S[i,k] = (self.V1c[i])*np.conjugate((self.V1c[i]-self.V1c[k])*self.Y1[i,k]);


# Decoupled Newton Raphson:
Explaination_by_Yash: Aforementioned solution method is in accord with  decoupled Newton Raphson. However, we need to erase coupling terms in jacobian for Q-δ and P-V. Furthermore, I dont wanna do everything again! I mean come on! Bachhe ki jaan lenge kya?   

In [83]:
class DC_NR():
  def __init__(self, Linedata1, Loaddata1, N=100, Epsilon=1e-5, S_base=100):
    self.Linedata1=Linedata1;
    self.Loaddata1=Loaddata1;
    self.N = N;
    self.Epsilon = Epsilon;
    self.S_base = 100; self.j = np.sqrt(-1+0j);
    self.Vmin = 0.95; self.Vmax = 1.05;
    self.Qmin = -1.0; self.Qmax = 1.0;
    self.P1 = np.copy(self.Loaddata1[:,3])/self.S_base;
    self.P1_cal = np.zeros(len(self.Loaddata1[:,0]));
    self.Q1 = np.copy(self.Loaddata1[:,4])/self.S_base;
    self.Q1_cal = np.zeros(len(self.Loaddata1[:,0]));
    self.Q1_old = np.copy(self.Loaddata1[:,4])/self.S_base;
    self.V1 = np.copy(self.Loaddata1[:,2]);
    self.V1_old = np.copy(self.Loaddata1[:,2]);
    self.Delta1 = np.zeros(len(self.Loaddata1[:,0]));
    self.Jacobian = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    self.I = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);
    self.S = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);
    self.S_loss = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);

  def common_member(self,a,b):
    a_set = set(a)
    b_set = set(b)

    # check length
    if len(a_set.intersection(b_set)) > 0:
        return(list(a_set.intersection(b_set)))
    else:
        return([])

  def Y_bus(self,Linedata1, Loaddata1):
    Y1= np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])],dtype=np.complex_);
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k)
      # print(f"This is i: {i} & this is k: {k}")
      Y1[i,k] += -1*(1/(self.Linedata1[self.common_member(np.where(self.Linedata1[:,0]==i)[0],np.where(self.Linedata1[:,1]==k)[0]),2]+self.j*self.Linedata1[self.common_member(np.where(self.Linedata1[:,0]==i)[0],np.where(self.Linedata1[:,1]==k)[0]),3]))[0]
    for i in self.Loaddata1[:,0]:
      i = int(i);
      Y1[i,i] += sum(1/(self.Linedata1[pp,2]+self.j*self.Linedata1[pp,3]) for pp in list(np.where(self.Linedata1[:,0]==i)[0]))
    G = np.real(Y1)
    B = np.imag(Y1)
    return G,B,Y1

  def Jacobian_Calculations(self):
    # H:
    self.H = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    for i,k in itt.product(self.Loaddata1[:,0],self.Loaddata1[:,0]):
      i = int(i); k = int(k);
      if self.Loaddata1[k,1] != 0 and self.Loaddata1[i,1] !=0:
        if i==k:
          self.H[i,k] = -np.real(self.Q1[i])-self.B[i,i]*(abs(self.V1[i])**2);
        else:
          self.H[i,k] =  abs(self.V1[i])*abs(self.V1[k])*(self.G[i,k]*np.sin(self.Delta1[i]-self.Delta1[k])-self.B[i,k]*np.cos(self.Delta1[i]-self.Delta1[k]))
    # N:
    self.N = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    # J:
    self.J = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    # L:
    self.L = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    for i,k in itt.product(self.Loaddata1[:,0],self.Loaddata1[:,0]):
      i = int(i); k = int(k);
      if self.Loaddata1[i,1] == 1 and self.Loaddata1[k,1] == 1:
        if i==k:
          self.L[i,k] = np.real(self.Q1[i])-self.B[i,i]*(abs(self.V1[i])**2);
        else:
          self.L[i,k] =  abs(self.V1[i])*abs(self.V1[k])*(self.G[i,k]*np.sin(self.Delta1[i]-self.Delta1[k])-self.B[i,k]*np.cos(self.Delta1[i]-self.Delta1[k]))
    # Formulation:
    # self.Jacobian[:len(self.Loaddata1[:,0])-1,:len(self.Loaddata1[:,0])-1] = self.H[1:len(self.Loaddata1[:,0]),1:len(self.Loaddata1[:,0])]
    # self.Jacobian[:len(self.Loaddata1[:,0])-1,np.where(self.Loaddata1[:,1]==1)[0]] = self.N[1:len(self.Loaddata1[:,0]),np.where(self.Loaddata1[:,1]==1)[0]]
    # self.Jacobian[np.where(self.Loaddata1[:,1]==1)[0],:len(self.Loaddata1[:,0])-1] = self.J[np.where(self.Loaddata1[:,1]==1)[0],1:len(self.Loaddata1[:,0])]
    # self.Jacobian[np.where(self.Loaddata1[:,1]==1)[0],np.where(self.Loaddata1[:,1]==1)[0]] = self.L[np.where(self.Loaddata1[:,1]==1)[0],np.where(self.Loaddata1[:,1]==1)[0]]
    self.Jacobian[0:2,0:2] = self.H[1:3,1:3]
    self.Jacobian[0:2,2] = self.N[1:3,1]
    self.Jacobian[2,0:2] = self.J[1,1:3]
    self.Jacobian[2,2] = self.L[1,1]
      
    return self.Jacobian

  def Calculations(self):
    self.G,self.B,self.Y1 = self.Y_bus(self.Linedata1,self.Loaddata1);
    Convergence = {};
    for ii in range(self.N):
      # print(f"Yeh hai ii: {ii}")
      for i in self.Loaddata1[:,0]:
        i = int(i)
        if self.Loaddata1[i,1] != 0:
          self.P1_cal[i] = abs(self.V1[i])*sum([self.V1[int(k)]*(self.G[i,int(k)]*np.cos(self.Delta1[i]-self.Delta1[int(k)])+self.B[i,int(k)]*np.sin(self.Delta1[i]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
        if self.Loaddata1[i,1] == 1:
          self.Q1_cal[i] = abs(self.V1[i])*sum([self.V1[int(k)]*(self.G[i,int(k)]*np.sin(self.Delta1[i]-self.Delta1[int(k)])-self.B[i,int(k)]*np.cos(self.Delta1[i]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
      self.Q1_cal[self.Q1_cal<=self.Qmin]=self.Qmin;
      self.Q1_cal[self.Q1_cal>=self.Qmax]=self.Qmax;
      self.Del_P1 = np.copy(self.P1)-np.copy(self.P1_cal);
      self.Del_Q1 = np.copy(self.Q1)-np.copy(self.Q1_cal);
      # print("Our Epsilon: ",self.Epsilon)
      # print(abs(self.Del_P1))
      # print(abs(self.Del_Q1))
      # print("np.all(abs(Del_P1[1:]) < self.Epsilon): ",np.all(abs(self.Del_P1[1:]) < self.Epsilon))
      # print("abs(Del_P1[1:]) < self.Epsilon: ",abs(self.Del_P1[1:]) < self.Epsilon)
      if np.all(abs(self.Del_P1[1:]) < self.Epsilon) and np.all(abs(self.Del_Q1[1:]) < self.Epsilon): #and np.all(abs(Del_Q1[1:]))<= self.Epsilon:
        # print(f"Yeh hai ii during break: {ii}")
        break
      # print("Reaching")
      self.delY = np.append(np.copy(self.Del_P1[1:]),(self.Del_Q1[np.where(self.Loaddata1[:,1] == 1)]))
      self.Jacobian = self.Jacobian_Calculations()
      self.delX = np.matmul(np.linalg.inv(self.Jacobian),self.delY)
      self.Delta1[np.where(self.Loaddata1[:,1]!=0)[0]] += self.delX[:len(np.where(self.Loaddata1[:,1]!=0)[0])]
      self.V1[np.where(self.Loaddata1[:,1]==1)[0]] += self.delX[np.where(self.Loaddata1[:,1]==1)[0]]*self.V1[np.where(self.Loaddata1[:,1]==1)[0]]
      self.V1[self.V1<=self.Vmin]=self.Vmin;
      self.V1[self.V1>=self.Vmax]=self.Vmax;
      Convergence[ii] = [(self.Del_P1[1:]), (self.Del_Q1[1:])];
    # print("Baba re baba agaya bahar on iteration: ",ii+1)
    # Slack bus computations:
    self.P1_cal[0] = abs(self.V1[0])*sum([self.V1[int(k)]*(self.G[0,int(k)]*np.cos(self.Delta1[0]-self.Delta1[int(k)])+self.B[0,int(k)]*np.sin(self.Delta1[0]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
    self.Q1_cal[0] = abs(self.V1[0])*sum([self.V1[int(k)]*(self.G[0,int(k)]*np.sin(self.Delta1[0]-self.Delta1[int(k)])-self.B[0,int(k)]*np.cos(self.Delta1[0]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])

    # Line Flows:
    ## Branch Currents:
    self.V1c = np.copy(self.V1)*(np.cos(self.Delta1)+self.j*np.sin(self.Delta1));
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.I[i,k] = (self.V1c[i]-self.V1c[k])*self.Y1[i,k];

    ## Branch Flow:
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.S[i,k] = (self.V1c[i])*np.conjugate((self.V1c[i]-self.V1c[k])*self.Y1[i,k]);

    ## Branch losses:
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.S_loss[i,k] = self.S[i,k]-self.S[k,i];

    return self.V1, self.Delta1, self.P1_cal, self.Q1_cal, self.Jacobian, self.I, self.S, self.S_loss, ii+1, self.V1c, self.Del_P1, self.Del_Q1, Convergence

In [84]:
NR = DC_NR(Linedata1, Loaddata1,N=100,Epsilon=1e-14)
Y = NR.Y_bus(Linedata1, Loaddata1)
V1, Delta1, P1_cal, Q1_cal, Jacobian, I, S, S_loss, N, V1c, Del_P1, Del_Q1, dir_Convergence = NR.Calculations()
print("V1: ",V1)
print("Delta1: ",Delta1)
print("P1_cal: ",P1_cal)
print("Q1_cal: ",Q1_cal)
print("Jacobian: ",Jacobian)
print("I: ",I)
print("S: ",S)
print("S_loss: ",S_loss)
print("N: ",N)
print("self.V1c: ",V1c)
print("Del_P1: ",Del_P1)
print("Del_Q1: ",Del_Q1)
print("abs(Del_P1): ",abs(Del_P1))
print("abs(Del_Q1): ",abs(Del_Q1))
print(dir_Convergence)

C:\Users\y.raghuwanshi\AppData\Local\Temp\ipykernel_11852\3134140389.py:123: ComplexWarning: Casting complex values to real discards the imaginary part
  self.I[i,k] = (self.V1c[i]-self.V1c[k])*self.Y1[i,k];
C:\Users\y.raghuwanshi\AppData\Local\Temp\ipykernel_11852\3134140389.py:128: ComplexWarning: Casting complex values to real discards the imaginary part
  self.S[i,k] = (self.V1c[i])*np.conjugate((self.V1c[i]-self.V1c[k])*self.Y1[i,k]);


V1:  [1.05       0.99195123 1.04      ]
Delta1:  [ 0.         -0.05502531 -0.00805849]
P1_cal:  [ 2.14027632 -4.          2.        ]
Q1_cal:  [ 0.90579166 -1.          0.        ]
Jacobian:  [[ 53.66629706 -32.20078069   0.        ]
 [-33.75068595  67.0592       0.        ]
 [  0.           0.          48.66629706]]
I:  [[ 0.         -1.68659865 -0.35175975]
 [ 1.68659865  0.          2.26983401]
 [ 0.35175975 -2.26983401  0.        ]]
S:  [[ 0.         -1.77092858 -0.36934774]
 [ 1.70570409  0.          2.29429591]
 [ 0.36763861 -2.36763861  0.        ]]
S_loss:  [[ 0.         -3.47663267 -0.73698635]
 [ 3.47663267  0.          4.66193452]
 [ 0.73698635 -4.66193452  0.        ]]
N:  100
self.V1c:  [1.05      +0.j         0.99044991-0.05455489j 1.03996623-0.00838074j]
Del_P1:  [0.00000000e+00 8.88178420e-16 3.33066907e-15]
Del_Q1:  [ 0.  -1.5  0. ]
abs(Del_P1):  [0.00000000e+00 8.88178420e-16 3.33066907e-15]
abs(Del_Q1):  [0.  1.5 0. ]
{0: [array([-2.86  ,  1.4384]), array([-1.5,  0. 

# Fast Decoupled Load Flow:
Explaination_by_Yash: The jacobian in fast decoupled load flow become constant after following assumptions:
1. |V| ~ 1.
2. X/R >> 1.
3. δ almost constant through out the system.  
4. Jaldi Jaldi Jacobian ko constant karke nipta deta hun... kal subah Panwel bhi nikalna hai...

In [85]:
class FDLF():
  def __init__(self, Linedata1, Loaddata1, N=100, Epsilon=1e-5, S_base=100):
    self.Linedata1=Linedata1;
    self.Loaddata1=Loaddata1;
    self.N0 = N;
    self.Epsilon = Epsilon;
    self.S_base = 100; self.j = np.sqrt(-1+0j);
    self.Vmin = 0.95; self.Vmax = 1.05;
    self.Qmin = -1.0; self.Qmax = 1.0;
    self.P1 = np.copy(self.Loaddata1[:,3])/self.S_base;
    self.P1_cal = np.zeros(len(self.Loaddata1[:,0]));
    self.Q1 = np.copy(self.Loaddata1[:,4])/self.S_base;
    self.Q1_cal = np.zeros(len(self.Loaddata1[:,0]));
    self.Q1_old = np.copy(self.Loaddata1[:,4])/self.S_base;
    self.V1 = np.copy(self.Loaddata1[:,2]);
    self.V1_old = np.copy(self.Loaddata1[:,2]);
    self.Delta1 = np.zeros(len(self.Loaddata1[:,0]));
    self.Jacobian = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    self.I = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);
    self.S = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);
    self.S_loss = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])]);

  def common_member(self,a,b):
    a_set = set(a)
    b_set = set(b)

    # check length
    if len(a_set.intersection(b_set)) > 0:
        return(list(a_set.intersection(b_set)))
    else:
        return([])

  def Y_bus(self,Linedata1, Loaddata1):
    Y1= np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])],dtype=np.complex_);
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k)
      # print(f"This is i: {i} & this is k: {k}")
      Y1[i,k] += -1*(1/(self.Linedata1[self.common_member(np.where(self.Linedata1[:,0]==i)[0],np.where(self.Linedata1[:,1]==k)[0]),2]+self.j*self.Linedata1[self.common_member(np.where(self.Linedata1[:,0]==i)[0],np.where(self.Linedata1[:,1]==k)[0]),3]))[0]
    for i in self.Loaddata1[:,0]:
      i = int(i);
      Y1[i,i] += sum(1/(self.Linedata1[pp,2]+self.j*self.Linedata1[pp,3]) for pp in list(np.where(self.Linedata1[:,0]==i)[0]))
    G = np.real(Y1)
    B = np.imag(Y1)
    return G,B,Y1

  def Jacobian_Calculations(self):
    # H:
    self.H = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    for i,k in itt.product(self.Loaddata1[:,0],self.Loaddata1[:,0]):
      i = int(i); k = int(k);
      if self.Loaddata1[k,1] != 0 and self.Loaddata1[i,1] !=0:
        if i==k:
          self.H[i,k] = -self.B[i,i];
        else:
          self.H[i,k] =  -self.B[i,k];
    # N:
    self.N = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    # J:
    self.J = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    # L:
    self.L = np.zeros([len(self.Loaddata1[:,0]),len(self.Loaddata1[:,0])])
    for i,k in itt.product(self.Loaddata1[:,0],self.Loaddata1[:,0]):
      i = int(i); k = int(k);
      if self.Loaddata1[i,1] == 1 and self.Loaddata1[k,1] == 1:
        if i==k:
          self.L[i,k] = -self.B[i,i];
        else:
          self.L[i,k] =  -self.B[i,k];
    # Formulation:
    # self.Jacobian[:len(self.Loaddata1[:,0])-1,:len(self.Loaddata1[:,0])-1] = self.H[1:len(self.Loaddata1[:,0]),1:len(self.Loaddata1[:,0])]
    # self.Jacobian[:len(self.Loaddata1[:,0])-1,np.where(self.Loaddata1[:,1]==1)[0]] = self.N[1:len(self.Loaddata1[:,0]),np.where(self.Loaddata1[:,1]==1)[0]]
    # self.Jacobian[np.where(self.Loaddata1[:,1]==1)[0],:len(self.Loaddata1[:,0])-1] = self.J[np.where(self.Loaddata1[:,1]==1)[0],1:len(self.Loaddata1[:,0])]
    # self.Jacobian[np.where(self.Loaddata1[:,1]==1)[0],np.where(self.Loaddata1[:,1]==1)[0]] = self.L[np.where(self.Loaddata1[:,1]==1)[0],np.where(self.Loaddata1[:,1]==1)[0]]
    self.Jacobian[0:2,0:2] = self.H[1:3,1:3]
    self.Jacobian[0:2,2] = self.N[1:3,1]
    self.Jacobian[2,0:2] = self.J[1,1:3]
    self.Jacobian[2,2] = self.L[1,1]
      
    return self.Jacobian

  def Calculations(self):
    self.G,self.B,self.Y1 = self.Y_bus(self.Linedata1,self.Loaddata1);
    self.Jacobian = self.Jacobian_Calculations()
    print(self.Jacobian)
    Convergence = {};
    print("self.N",self.N)
    for ii in range(self.N0):
      # print(f"Yeh hai ii: {ii}")
      for i in self.Loaddata1[:,0]:
        i = int(i)
        if self.Loaddata1[i,1] != 0:
          self.P1_cal[i] = abs(self.V1[i])*sum([self.V1[int(k)]*(self.G[i,int(k)]*np.cos(self.Delta1[i]-self.Delta1[int(k)])+self.B[i,int(k)]*np.sin(self.Delta1[i]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
        if self.Loaddata1[i,1] == 1:
          self.Q1_cal[i] = abs(self.V1[i])*sum([self.V1[int(k)]*(self.G[i,int(k)]*np.sin(self.Delta1[i]-self.Delta1[int(k)])-self.B[i,int(k)]*np.cos(self.Delta1[i]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
      self.Q1_cal[self.Q1_cal<=self.Qmin]=self.Qmin;
      self.Q1_cal[self.Q1_cal>=self.Qmax]=self.Qmax;
      self.Del_P1 = np.copy(self.P1)-np.copy(self.P1_cal);
      self.Del_Q1 = np.copy(self.Q1)-np.copy(self.Q1_cal);
      # print("Our Epsilon: ",self.Epsilon)
      # print(abs(self.Del_P1))
      # print(abs(self.Del_Q1))
      # print("np.all(abs(Del_P1[1:]) < self.Epsilon): ",np.all(abs(self.Del_P1[1:]) < self.Epsilon))
      # print("abs(Del_P1[1:]) < self.Epsilon: ",abs(self.Del_P1[1:]) < self.Epsilon)
      if np.all(abs(self.Del_P1[1:]) < self.Epsilon) and np.all(abs(self.Del_Q1[1:]) < self.Epsilon): #and np.all(abs(Del_Q1[1:]))<= self.Epsilon:
        # print(f"Yeh hai ii during break: {ii}")
        break
      # print("Reaching")
      self.delY = np.append(np.copy(self.Del_P1[1:])/np.copy(self.V1[1:]),(self.Del_Q1[np.where(self.Loaddata1[:,1] == 1)]/np.copy(self.V1[np.where(self.Loaddata1[:,1] == 1)])))
      self.delX = np.matmul(np.linalg.inv(self.Jacobian),self.delY)
      self.Delta1[np.where(self.Loaddata1[:,1]!=0)[0]] += self.delX[:len(np.where(self.Loaddata1[:,1]!=0)[0])]
      self.V1[np.where(self.Loaddata1[:,1]==1)[0]] += self.delX[np.where(self.Loaddata1[:,1]==1)[0]]*self.V1[np.where(self.Loaddata1[:,1]==1)[0]]
      self.V1[self.V1<=self.Vmin]=self.Vmin;
      self.V1[self.V1>=self.Vmax]=self.Vmax;
      Convergence[ii] = [(self.Del_P1[1:]), (self.Del_Q1[1:])];
    # print("Baba re baba agaya bahar on iteration: ",ii+1)
    # Slack bus computations:
    self.P1_cal[0] = abs(self.V1[0])*sum([self.V1[int(k)]*(self.G[0,int(k)]*np.cos(self.Delta1[0]-self.Delta1[int(k)])+self.B[0,int(k)]*np.sin(self.Delta1[0]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])
    self.Q1_cal[0] = abs(self.V1[0])*sum([self.V1[int(k)]*(self.G[0,int(k)]*np.sin(self.Delta1[0]-self.Delta1[int(k)])-self.B[0,int(k)]*np.cos(self.Delta1[0]-self.Delta1[int(k)])) for k in self.Loaddata1[:,0]])

    # Line Flows:
    ## Branch Currents:
    self.V1c = np.copy(self.V1)*(np.cos(self.Delta1)+self.j*np.sin(self.Delta1));
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.I[i,k] = (self.V1c[i]-self.V1c[k])*self.Y1[i,k];

    ## Branch Flow:
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.S[i,k] = (self.V1c[i])*np.conjugate((self.V1c[i]-self.V1c[k])*self.Y1[i,k]);

    ## Branch losses:
    for i,k in zip(list(self.Linedata1[:,0]),list(self.Linedata1[:,1])):
      i = int(i);k=int(k);
      self.S_loss[i,k] = self.S[i,k]-self.S[k,i];

    return self.V1, self.Delta1, self.P1_cal, self.Q1_cal, self.Jacobian, self.I, self.S, self.S_loss, ii+1, self.V1c, self.Del_P1, self.Del_Q1, Convergence

In [86]:
FDLF = FDLF(Linedata1, Loaddata1,N=100,Epsilon=1e-14)
Y = FDLF.Y_bus(Linedata1, Loaddata1)
V1, Delta1, P1_cal, Q1_cal, Jacobian, I, S, S_loss, N, V1c, Del_P1, Del_Q1, dir_Convergence = FDLF.Calculations()
print("V1: ",V1)
print("Delta1: ",Delta1)
print("P1_cal: ",P1_cal)
print("Q1_cal: ",Q1_cal)
print("Jacobian: ",Jacobian)
print("I: ",I)
print("S: ",S)
print("S_loss: ",S_loss)
print("N: ",N)
print("self.V1c: ",V1c)
print("Del_P1: ",Del_P1)
print("Del_Q1: ",Del_Q1)
print("abs(Del_P1): ",abs(Del_P1))
print("abs(Del_Q1): ",abs(Del_Q1))
print(dir_Convergence)

[[ 52. -32.   0.]
 [-32.  62.   0.]
 [  0.   0.  52.]]
self.N [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
V1:  [1.05       0.99193352 1.04      ]
Delta1:  [ 0.         -0.05501821 -0.0080589 ]
P1_cal:  [ 2.14030322 -4.          2.        ]
Q1_cal:  [ 0.90623476 -1.          0.        ]
Jacobian:  [[ 52. -32.   0.]
 [-32.  62.   0.]
 [  0.   0.  52.]]
I:  [[ 0.         -1.68661151 -0.35177252]
 [ 1.68661151  0.          2.26984094]
 [ 0.35177252 -2.26984094  0.        ]]
S:  [[ 0.         -1.77094208 -0.36936114]
 [ 1.70570571  0.          2.29429429]
 [ 0.36765194 -2.36765194  0.        ]]
S_loss:  [[ 0.         -3.4766478  -0.73701309]
 [ 3.4766478   0.          4.66194623]
 [ 0.73701309 -4.66194623  0.        ]]
N:  100
self.V1c:  [1.05      +0.j         0.9904326 -0.05454688j 1.03996623-0.00838116j]
Del_P1:  [ 0.00000000e+00  5.32907052e-15 -3.99680289e-15]
Del_Q1:  [ 0.  -1.5  0. ]
abs(Del_P1):  [0.00000000e+00 5.32907052e-15 3.99680289e-15]
abs(Del_Q1):  [0.  1.5 0. ]
{0: [array([-2.86  

C:\Users\y.raghuwanshi\AppData\Local\Temp\ipykernel_11852\799588279.py:125: ComplexWarning: Casting complex values to real discards the imaginary part
  self.I[i,k] = (self.V1c[i]-self.V1c[k])*self.Y1[i,k];
C:\Users\y.raghuwanshi\AppData\Local\Temp\ipykernel_11852\799588279.py:130: ComplexWarning: Casting complex values to real discards the imaginary part
  self.S[i,k] = (self.V1c[i])*np.conjugate((self.V1c[i]-self.V1c[k])*self.Y1[i,k]);


# DC Load Flow:
Explaination_by_Yash: We need to change the way we define Y-bus matrix for DC load flow. We just need the reactance from linedata now.

## Formulation of B-bus matrix

In [87]:
def common_member(a,b):
  a_set = set(a)
  b_set = set(b)

  # check length
  if len(a_set.intersection(b_set)) > 0:
      return(list(a_set.intersection(b_set)))
  else:
      return([])

B_bus= np.zeros([len(Loaddata1[:,0]),len(Loaddata1[:,0])],dtype=np.complex_);
for i,k in zip(list(Linedata1[:,0]),list(Linedata1[:,1])):
  i = int(i);k=int(k)
  # print(f"This is i: {i} & this is k: {k}")
  B_bus[i,k] += 1*(1/(j*Linedata1[common_member(np.where(Linedata1[:,0]==i)[0],np.where(Linedata1[:,1]==k)[0]),3]))[0]
for i in Loaddata1[:,0]:
  i = int(i);
  B_bus[i,i] += -sum(1/(j*Linedata1[pp,3]) for pp in list(np.where(Linedata1[:,0]==i)[0]))
G = np.real(B_bus)
B = np.imag(B_bus)
# print(B_bus)
# print(G)
print(B)

[[ 58.33333333 -25.         -33.33333333]
 [-25.          65.         -40.        ]
 [-33.33333333 -40.          73.33333333]]


Lastly one can do simple Equation solving using Sympy library's solver.